# Project 3 - Part 2 (Feature Engineering)
## Roman Sielewicz, Imani Mufti, Alek Carlson, Adam Tassabehji, Sharath Reddy
### 2021 April 29

### Importing Packages

In [1]:
# Importing Packages
from datetime import datetime
start_time = datetime.now()
# import kaleido
import datetime as dt
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import log10
import re
import statistics
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
# # packages for google colab
# # Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [3]:
# # getting link and accessing data
# # link: shareable link to excel file
# link = 'https://drive.google.com/file/d/1WuK0lj-z0qnILlDbx9VXaLM-Ee-80mTs/view?usp=sharing' 
# id = link.split('/')[-2] # id: the drive file identifier
# print(id)

In [4]:
# # creating pandas dataframe
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('card_transactions_clean.csv')  
# df = pd.read_csv('card_transactions_clean.csv')

In [5]:
# Alternative for running on local machine (uncomment to run)
# Loading Data
df = pd.read_csv('card_transactions_clean.csv')

In [6]:
# Information about the data
print(f'{df.shape[0]} \trows \n{df.shape[1]} \tcolumns')
# print(df.dtypes)
df.head()

96397 	rows 
10 	columns


,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [7]:
#DOW Risk Variable (1)

#date to datetime
df['Date'] = df['Date'].astype('datetime64[ns]')

#get weekday
df['dow'] = df.Date.dt.day_name()

#train/test splitting
train_test = df[df.Date < '2010-11-01']

#parameter for smoothing function
c=4 
#parameter for smoothing function
nmid=20 
#average target value
y_avg = train_test['Fraud'].mean()
#average target by dow
y_dow = train_test.groupby('dow')['Fraud'].mean()
#number of instances for each dow
num_instances_dow = train_test.groupby('dow').size()
#applying smoothing function
y_dow_smooth = y_avg + (y_dow - y_avg) / (1 + np.exp(-(num_instances_dow - nmid)/c))

#final target encoded variable
df['dow_risk'] = df['dow'].map(y_dow_smooth)

#drop unnecessary dow column
df.drop(columns='dow',inplace=True)

df.head()

,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud,dow_risk
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0.025994
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,0.025994
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,0.025994
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,0.025994
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0.025994


In [8]:
# Creating new entity groupings based on class 10 slides (3 new groupings and we will use Cardnum + Merchnum)

#card at this merchant
df['card_merchant'] = df.Cardnum.astype(str) + df.Merchnum.astype(str)

#card in this zip
df['card_zip'] = df.Cardnum.astype(str) + df.Merchzip.astype(str)

#card in this state
df['card_state'] = df.Cardnum.astype(str) + df.Merchstate.astype(str)

df.head()

,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud,dow_risk,card_merchant,card_zip,card_state
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0.025994,51421904395509006296254,514219043938118.0,5142190439TN
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,0.025994,514218397361003026333,51421839731803.0,5142183973MA
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,0.025994,51421317214503082993600,514213172120706.0,5142131721MD
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,0.025994,51421484525509006296254,514214845238118.0,5142148452TN
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0.025994,51421904395509006296254,514219043938118.0,5142190439TN


In [9]:
#Prepping for Variable Creation

entities = ['Cardnum','Merchnum','card_merchant','card_zip','card_state']
timeframes = [0,1,3,7,14,30]

df1 = df.copy()
final = df.copy()
df1['check_date'] = df1.Date
df1['check_record'] = df1.Recnum

In [10]:
#Days Since Variables (5)
vars_before = len(final.columns)
for entity in entities:
    df_1 = df1[['Recnum','Date',entity]]
    df_r = df1[['check_record','check_date',entity]]
    temp = pd.merge(df_1,df_r, left_on = entity, right_on = entity)
    
    #days since
    day_since_df = temp[temp.Recnum > temp.check_record][['Recnum','Date','check_date']] \
        .groupby('Recnum')[['Date','check_date']].last()
    mapper = (day_since_df.Date - day_since_df.check_date).dt.days
    final[entity + '_day_since'] = final.Recnum.map(mapper)
    final[entity + '_day_since'].fillna(365, inplace = True)
    print(f'\n{entity}_day_since ---> Done')
## counting number of variables made
vars_after = len(final.columns)
print(f'Variables Before:\t{vars_before}')
print(f'Variables After:\t{vars_after}')
print(f'Vars Created:\t{vars_after-vars_before}')


Cardnum_day_since ---> Done

Merchnum_day_since ---> Done

card_merchant_day_since ---> Done

card_zip_day_since ---> Done

card_state_day_since ---> Done
Variables Before:	14
Variables After:	19
Vars Created:	5


In [11]:
#Frequency Variables (30)
vars_before = len(final.columns)

for entity in entities:
    df_1 = df1[['Recnum','Date',entity]]
    df_r = df1[['check_record','check_date',entity]]
    temp = pd.merge(df_1,df_r, left_on = entity, right_on = entity)

    #velocity from proj 2
    for offset_t in timeframes:
        count_day_df = temp[(temp.check_date >= (temp.Date - dt.timedelta(offset_t)))
                           & (temp.Recnum >= temp.check_record)]
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_df.groupby('Recnum')[entity].count()
        final[col_name] = final.Recnum.map(mapper2)
        
        print(f'{entity}_count_{str(offset_t)} ---> Done')
        
## counting number of variables made
vars_after = len(final.columns)
print(f'Variables Before:\t{vars_before}')
print(f'Variables After:\t{vars_after}')
print(f'Vars Created:\t{vars_after-vars_before}')

Cardnum_count_0 ---> Done
Cardnum_count_1 ---> Done
Cardnum_count_3 ---> Done
Cardnum_count_7 ---> Done
Cardnum_count_14 ---> Done
Cardnum_count_30 ---> Done
Merchnum_count_0 ---> Done
Merchnum_count_1 ---> Done
Merchnum_count_3 ---> Done
Merchnum_count_7 ---> Done
Merchnum_count_14 ---> Done
Merchnum_count_30 ---> Done
card_merchant_count_0 ---> Done
card_merchant_count_1 ---> Done
card_merchant_count_3 ---> Done
card_merchant_count_7 ---> Done
card_merchant_count_14 ---> Done
card_merchant_count_30 ---> Done
card_zip_count_0 ---> Done
card_zip_count_1 ---> Done
card_zip_count_3 ---> Done
card_zip_count_7 ---> Done
card_zip_count_14 ---> Done
card_zip_count_30 ---> Done
card_state_count_0 ---> Done
card_state_count_1 ---> Done
card_state_count_3 ---> Done
card_state_count_7 ---> Done
card_state_count_14 ---> Done
card_state_count_30 ---> Done
Variables Before:	19
Variables After:	49
Vars Created:	30


In [12]:
#Relative Velocity (40)
vars_before = len(final.columns)

for entity in entities:
    for d in ['0','1']:
        for dd in ['3','7','14','30']:
            final[entity + '_count_' + d + dd] \
            =final[entity + '_count_' + d] / (final[entity+'_count_'+dd] / float(dd))
            print(f'{entity}_count_{d}{dd} ---> Done')

## counting number of variables made
vars_after = len(final.columns)
print(f'Variables Before:\t{vars_before}')
print(f'Variables After:\t{vars_after}')
print(f'Vars Created:\t{vars_after-vars_before}')

Cardnum_count_03 ---> Done
Cardnum_count_07 ---> Done
Cardnum_count_014 ---> Done
Cardnum_count_030 ---> Done
Cardnum_count_13 ---> Done
Cardnum_count_17 ---> Done
Cardnum_count_114 ---> Done
Cardnum_count_130 ---> Done
Merchnum_count_03 ---> Done
Merchnum_count_07 ---> Done
Merchnum_count_014 ---> Done
Merchnum_count_030 ---> Done
Merchnum_count_13 ---> Done
Merchnum_count_17 ---> Done
Merchnum_count_114 ---> Done
Merchnum_count_130 ---> Done
card_merchant_count_03 ---> Done
card_merchant_count_07 ---> Done
card_merchant_count_014 ---> Done
card_merchant_count_030 ---> Done
card_merchant_count_13 ---> Done
card_merchant_count_17 ---> Done
card_merchant_count_114 ---> Done
card_merchant_count_130 ---> Done
card_zip_count_03 ---> Done
card_zip_count_07 ---> Done
card_zip_count_014 ---> Done
card_zip_count_030 ---> Done
card_zip_count_13 ---> Done
card_zip_count_17 ---> Done
card_zip_count_114 ---> Done
card_zip_count_130 ---> Done
card_state_count_03 ---> Done
card_state_count_07 ---> D

In [13]:
final.head()

,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud,dow_risk,card_merchant,card_zip,card_state,Cardnum_day_since,Merchnum_day_since,card_merchant_day_since,card_zip_day_since,card_state_day_since,Cardnum_count_0,Cardnum_count_1,Cardnum_count_3,Cardnum_count_7,Cardnum_count_14,Cardnum_count_30,Merchnum_count_0,Merchnum_count_1,Merchnum_count_3,Merchnum_count_7,Merchnum_count_14,Merchnum_count_30,card_merchant_count_0,card_merchant_count_1,card_merchant_count_3,card_merchant_count_7,card_merchant_count_14,card_merchant_count_30,card_zip_count_0,card_zip_count_1,card_zip_count_3,card_zip_count_7,card_zip_count_14,card_zip_count_30,card_state_count_0,card_state_count_1,card_state_count_3,card_state_count_7,card_state_count_14,card_state_count_30,Cardnum_count_03,Cardnum_count_07,Cardnum_count_014,Cardnum_count_030,Cardnum_count_13,Cardnum_count_17,Cardnum_count_114,Cardnum_count_130,Merchnum_count_03,Merchnum_count_07,Merchnum_count_014,Merchnum_count_030,Merchnum_count_13,Merchnum_count_17,Merchnum_count_114,Merchnum_count_130,card_merchant_count_03,card_merchant_count_07,card_merchant_count_014,card_merchant_count_030,card_merchant_count_13,card_merchant_count_17,card_merchant_count_114,card_merchant_count_130,card_zip_count_03,card_zip_count_07,card_zip_count_014,card_zip_count_030,card_zip_count_13,card_zip_count_17,card_zip_count_114,card_zip_count_130,card_state_count_03,card_state_count_07,card_state_count_014,card_state_count_030,card_state_count_13,card_state_count_17,card_state_count_114,card_state_count_130
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0.025994,51421904395509006296254,514219043938118.0,5142190439TN,365.0,365.0,365.0,365.0,365.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,0.025994,514218397361003026333,51421839731803.0,5142183973MA,365.0,365.0,365.0,365.0,365.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,0.025994,51421317214503082993600,514213172120706.0,5142131721MD,365.0,365.0,365.0,365.0,365.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,0.025994,51421484525509006296254,514214845238118.0,5142148452TN,365.0,0.0,365.0,365.0,365.0,1,1,1,1,1,1,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0.025994,51421904395509006296254,514219043938118.0,5142190439TN,0.0,0.0,0.0,0.0,0.0,2,2,2,2,2,2,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0


**Amount Variables**

In [14]:
## Kernel keeps crashing running out of memory. Maybe run in the local machine 
##5*8*6=240 variables
vars_before = len(final.columns)

amount_start_time=datetime.now()

for entity in entities:
    df_1=df1[['Recnum','Date','Amount',entity]]
    df_r=df1[['check_record','check_date','Amount',entity]]
    temp=pd.merge(df_1,df_r,left_on=entity,right_on=entity)
    
    ## Velocity
    for offset_t in [0,1,3,7,14,30]:
        amount_day_df=temp[(temp.check_date>=(temp.Date-dt.timedelta(offset_t)))
                         & (temp.Recnum>temp.check_record)]
        ##mean
        col_name_mean=f'{entity}_mean_{offset_t}'
        mapper_mean=amount_day_df.groupby(['Recnum',entity])['Amount_y'].mean().reset_index().drop(columns=[entity])
        mapper_mean.rename(columns={'Amount_y':col_name_mean},inplace=True)
        final = final.merge(mapper_mean, left_on="Recnum", right_on="Recnum", how="left")
        final[col_name_mean] = final[col_name_mean].fillna(0)
        ##amount/mean
        actual = entity+"_actual/"+"mean_"+str(offset_t)
        final_temp=final.loc[final[col_name_mean]!=0,["Recnum", "Amount", col_name_mean]]
        print(final_temp.shape)
        final_temp[actual] = final_temp["Amount"] / final_temp[col_name_mean]
        final_temp = final_temp.drop(columns=["Amount", col_name_mean])
        final_temp[actual] = final_temp[actual].fillna(0)
        final = final.merge(final_temp, on="Recnum", how="left")
        print(f'{entity}_mean_{str(offset_t)}--->Done')
        print(f'{entity}actual/mean_{str(offset_t)}--->Done')
        
        ##max
        col_name_max=f'{entity}_max_{offset_t}'
        mapper_max=amount_day_df.groupby(['Recnum',entity])['Amount_y'].max().reset_index().drop(columns=[entity])
        mapper_max.rename(columns={'Amount_y':col_name_max},inplace=True)
        final = final.merge(mapper_max, left_on="Recnum", right_on="Recnum", how="left")
        final[col_name_max] = final[col_name_max].fillna(0)
        ##actual/max
        actual = entity+"_actual/"+"max_"+str(offset_t)
        print(actual)
        final_temp=final.loc[final[col_name_max]!=0,["Recnum", "Amount", col_name_max]]
        print(final_temp.shape)
        final_temp[actual] = final_temp["Amount"] / final_temp[col_name_max]
        final_temp = final_temp.drop(columns=["Amount", col_name_max])
        final_temp[actual] = final_temp[actual].fillna(0)
        final = final.merge(final_temp, on="Recnum", how="left")
        print(f'{entity}_max_{str(offset_t)}--->Done')
        print(f'{entity}actual/max_{str(offset_t)}--->Done')
        
        ##Median
        col_name_median=f'{entity}_median_{offset_t}'
        mapper_median=amount_day_df.groupby(['Recnum',entity])['Amount_y'].median().reset_index().drop(columns=[entity])
        mapper_median.rename(columns={'Amount_y':col_name_median},inplace=True)
        final = final.merge(mapper_median, left_on="Recnum", right_on="Recnum", how="left")
        final[col_name_median] = final[col_name_median].fillna(0)
        ##actual/median
        actual = entity+"_actual/"+"median_"+str(offset_t)
        print(actual)
        final_temp=final.loc[final[col_name_median]!=0,["Recnum", "Amount", col_name_median]]
        print(final_temp.shape)
        final_temp[actual] = final_temp["Amount"] / final_temp[col_name_median]
        final_temp = final_temp.drop(columns=["Amount", col_name_median])
        final_temp[actual] = final_temp[actual].fillna(0)
        final = final.merge(final_temp, on="Recnum", how="left")
        print(f'{entity}_median_{str(offset_t)}--->Done')
        print(f'{entity}actual/median_{str(offset_t)}--->Done')
        
        ##Total
        col_name_tot=f'{entity}_total_{offset_t}'
        mapper_total=amount_day_df.groupby(['Recnum',entity])['Amount_y'].sum().reset_index().drop(columns=[entity])
        mapper_total.rename(columns={'Amount_y':col_name_tot},inplace=True)
        final = final.merge(mapper_total, left_on="Recnum", right_on="Recnum", how="left")
        final[col_name_tot] = final[col_name_tot].fillna(0)
        ##actual/total
        actual = entity+"_actual/"+"total_"+str(offset_t)
        print(actual)
        final_temp=final.loc[final[col_name_tot]!=0,["Recnum", "Amount", col_name_tot]]
        print(final_temp.shape)
        final_temp[actual] = final_temp["Amount"] / final_temp[col_name_tot]
        final_temp = final_temp.drop(columns=["Amount", col_name_tot])
        final_temp[actual] = final_temp[actual].fillna(0)
        final = final.merge(final_temp, on="Recnum", how="left")
        
        print(f'{entity}_total_{str(offset_t)}--->Done')
        print(f'{entity}actual/total_{str(offset_t)}--->Done')
        
        
    
        
amount_end_time = datetime.now()
print(amount_start_time)
print(amount_end_time)  
print("The time taken to create all variables is:")
print(amount_end_time - amount_start_time)

## counting number of variables made
vars_after = len(final.columns)
print(f'Variables Before:\t{vars_before}')
print(f'Variables After:\t{vars_after}')
print(f'Vars Created:\t{vars_after-vars_before}')

(34492, 3)
Cardnum_mean_0--->Done
Cardnumactual/mean_0--->Done
Cardnum_actual/max_0
(34492, 3)
Cardnum_max_0--->Done
Cardnumactual/max_0--->Done
Cardnum_actual/median_0
(34492, 3)
Cardnum_median_0--->Done
Cardnumactual/median_0--->Done
Cardnum_actual/total_0
(34492, 3)
Cardnum_total_0--->Done
Cardnumactual/total_0--->Done
(52928, 3)
Cardnum_mean_1--->Done
Cardnumactual/mean_1--->Done
Cardnum_actual/max_1
(52928, 3)
Cardnum_max_1--->Done
Cardnumactual/max_1--->Done
Cardnum_actual/median_1
(52928, 3)
Cardnum_median_1--->Done
Cardnumactual/median_1--->Done
Cardnum_actual/total_1
(52928, 3)
Cardnum_total_1--->Done
Cardnumactual/total_1--->Done
(66735, 3)
Cardnum_mean_3--->Done
Cardnumactual/mean_3--->Done
Cardnum_actual/max_3
(66735, 3)
Cardnum_max_3--->Done
Cardnumactual/max_3--->Done
Cardnum_actual/median_3
(66735, 3)
Cardnum_median_3--->Done
Cardnumactual/median_3--->Done
Cardnum_actual/total_3
(66735, 3)
Cardnum_total_3--->Done
Cardnumactual/total_3--->Done
(80975, 3)
Cardnum_mean_7---

(46102, 3)
card_zip_median_30--->Done
card_zipactual/median_30--->Done
card_zip_actual/total_30
(46102, 3)
card_zip_total_30--->Done
card_zipactual/total_30--->Done
(20221, 3)
card_state_mean_0--->Done
card_stateactual/mean_0--->Done
card_state_actual/max_0
(20221, 3)
card_state_max_0--->Done
card_stateactual/max_0--->Done
card_state_actual/median_0
(20221, 3)
card_state_median_0--->Done
card_stateactual/median_0--->Done
card_state_actual/total_0
(20221, 3)
card_state_total_0--->Done
card_stateactual/total_0--->Done
(27582, 3)
card_state_mean_1--->Done
card_stateactual/mean_1--->Done
card_state_actual/max_1
(27582, 3)
card_state_max_1--->Done
card_stateactual/max_1--->Done
card_state_actual/median_1
(27582, 3)
card_state_median_1--->Done
card_stateactual/median_1--->Done
card_state_actual/total_1
(27582, 3)
card_state_total_1--->Done
card_stateactual/total_1--->Done
(35171, 3)
card_state_mean_3--->Done
card_stateactual/mean_3--->Done
card_state_actual/max_3
(35171, 3)
card_state_max_3-

In [15]:
# #Amount Variables (120)

# HAVING TROUBLE WITH THESE ONES, FOLLOWING CODE NOT WORKING - AC

# for entity in entities:
#     df_1 = df1[['Recnum','Date','Amount',entity]]
#     df_r = df1[['check_record','check_date','Amount',entity]]
#     temp = pd.merge(df_1,df_r, left_on = entity, right_on = entity)

#     for offset_t in timeframes:
#         count_day_df = temp[(temp.check_date >= (temp.Date - dt.timedelta(offset_t)))
#                            & (temp.Recnum >= temp.check_record)]
#         mapper_avg = count_day_df.groupby('Recnum')[entity,'Amount_x'].mean()
#         final[f'{entity}_avg_{offset_t}'] = final.Recnum.map(mapper_avg)
#         mapper_max = count_day_df.groupby('Recnum')[entity,'Amount_x'].max()
#         final[f'{entity}_max_{offset_t}'] = final.Recnum.map(mapper_max)
#         mapper_med = count_day_df.groupby('Recnum')[entity,'Amount_x'].median()
#         final[f'{entity}_med_{offset_t}'] = final.Recnum.map(mapper_med)
#         mapper_sum = count_day_df.groupby('Recnum')[entity,'Amount_x'].sum()
#         final[f'{entity}_total_{offset_t}'] = final.Recnum.map(mapper_sum)

#         print(f'{entity}_count_{str(offset_t)} ---> Done')

In [16]:
print(entities)

['Cardnum', 'Merchnum', 'card_merchant', 'card_zip', 'card_state']


In [17]:
#ONCE AMOUNT VARIABLES ARE CALCULATED NEED TO DO RELATIVE VELOCITY AGAIN FOR AMOUNT
# Relative Velocity
vars_before = len(final.columns)

metrics = ['_mean_', '_total_' ]
for entity in entities:
    for d in ['0','1']:
        for dd in ['3','7','14','30']:
            for metric in metrics:
                final[entity + metric + d + dd] \
                =final[entity + metric + d] / (final[entity+metric+dd] / float(dd))
                print(f'{entity}{metric}{d}{dd} ---> Done')
                
## counting number of variables made
vars_after = len(final.columns)
print(f'Variables Before:\t{vars_before}')
print(f'Variables After:\t{vars_after}')
print(f'Vars Created:\t{vars_after-vars_before}')

Cardnum_mean_03 ---> Done
Cardnum_total_03 ---> Done
Cardnum_mean_07 ---> Done
Cardnum_total_07 ---> Done
Cardnum_mean_014 ---> Done
Cardnum_total_014 ---> Done
Cardnum_mean_030 ---> Done
Cardnum_total_030 ---> Done
Cardnum_mean_13 ---> Done
Cardnum_total_13 ---> Done
Cardnum_mean_17 ---> Done
Cardnum_total_17 ---> Done
Cardnum_mean_114 ---> Done
Cardnum_total_114 ---> Done
Cardnum_mean_130 ---> Done
Cardnum_total_130 ---> Done
Merchnum_mean_03 ---> Done
Merchnum_total_03 ---> Done
Merchnum_mean_07 ---> Done
Merchnum_total_07 ---> Done
Merchnum_mean_014 ---> Done
Merchnum_total_014 ---> Done
Merchnum_mean_030 ---> Done
Merchnum_total_030 ---> Done
Merchnum_mean_13 ---> Done
Merchnum_total_13 ---> Done
Merchnum_mean_17 ---> Done
Merchnum_total_17 ---> Done
Merchnum_mean_114 ---> Done
Merchnum_total_114 ---> Done
Merchnum_mean_130 ---> Done
Merchnum_total_130 ---> Done
card_merchant_mean_03 ---> Done
card_merchant_total_03 ---> Done
card_merchant_mean_07 ---> Done
card_merchant_total_07 

In [18]:
print(f'{final.shape[0]} \trows \n{final.shape[1]} \tcolumns')
final.head()

96397 	rows 
409 	columns


,Recnum,Cardnum,Date,Merchnum,Merchdescription,Merchstate,Merchzip,Transtype,Amount,Fraud,dow_risk,card_merchant,card_zip,card_state,Cardnum_day_since,Merchnum_day_since,card_merchant_day_since,card_zip_day_since,card_state_day_since,Cardnum_count_0,Cardnum_count_1,Cardnum_count_3,Cardnum_count_7,Cardnum_count_14,Cardnum_count_30,Merchnum_count_0,Merchnum_count_1,Merchnum_count_3,Merchnum_count_7,Merchnum_count_14,Merchnum_count_30,card_merchant_count_0,card_merchant_count_1,card_merchant_count_3,card_merchant_count_7,card_merchant_count_14,card_merchant_count_30,card_zip_count_0,card_zip_count_1,card_zip_count_3,card_zip_count_7,card_zip_count_14,card_zip_count_30,card_state_count_0,card_state_count_1,card_state_count_3,card_state_count_7,card_state_count_14,card_state_count_30,Cardnum_count_03,Cardnum_count_07,Cardnum_count_014,Cardnum_count_030,Cardnum_count_13,Cardnum_count_17,Cardnum_count_114,Cardnum_count_130,Merchnum_count_03,Merchnum_count_07,Merchnum_count_014,Merchnum_count_030,Merchnum_count_13,Merchnum_count_17,Merchnum_count_114,Merchnum_count_130,card_merchant_count_03,card_merchant_count_07,card_merchant_count_014,card_merchant_count_030,card_merchant_count_13,card_merchant_count_17,card_merchant_count_114,card_merchant_count_130,card_zip_count_03,card_zip_count_07,card_zip_count_014,card_zip_count_030,card_zip_count_13,card_zip_count_17,card_zip_count_114,card_zip_count_130,card_state_count_03,card_state_count_07,card_state_count_014,card_state_count_030,card_state_count_13,card_state_count_17,card_state_count_114,card_state_count_130,Cardnum_mean_0,Cardnum_actual/mean_0,Cardnum_max_0,Cardnum_actual/max_0,Cardnum_median_0,Cardnum_actual/median_0,Cardnum_total_0,Cardnum_actual/total_0,Cardnum_mean_1,Cardnum_actual/mean_1,Cardnum_max_1,Cardnum_actual/max_1,Cardnum_median_1,Cardnum_actual/median_1,Cardnum_total_1,Cardnum_actual/total_1,Cardnum_mean_3,Cardnum_actual/mean_3,Cardnum_max_3,Cardnum_actual/max_3,Cardnum_median_3,Cardnum_actual/median_3,Cardnum_total_3,Cardnum_actual/total_3,Cardnum_mean_7,Cardnum_actual/mean_7,Cardnum_max_7,Cardnum_actual/max_7,Cardnum_median_7,Cardnum_actual/median_7,Cardnum_total_7,Cardnum_actual/total_7,Cardnum_mean_14,Cardnum_actual/mean_14,Cardnum_max_14,Cardnum_actual/max_14,Cardnum_median_14,Cardnum_actual/median_14,Cardnum_total_14,Cardnum_actual/total_14,Cardnum_mean_30,Cardnum_actual/mean_30,Cardnum_max_30,Cardnum_actual/max_30,Cardnum_median_30,Cardnum_actual/median_30,Cardnum_total_30,Cardnum_actual/total_30,Merchnum_mean_0,Merchnum_actual/mean_0,Merchnum_max_0,Merchnum_actual/max_0,Merchnum_median_0,Merchnum_actual/median_0,Merchnum_total_0,Merchnum_actual/total_0,Merchnum_mean_1,Merchnum_actual/mean_1,Merchnum_max_1,Merchnum_actual/max_1,Merchnum_median_1,Merchnum_actual/median_1,Merchnum_total_1,Merchnum_actual/total_1,Merchnum_mean_3,Merchnum_actual/mean_3,Merchnum_max_3,Merchnum_actual/max_3,Merchnum_median_3,Merchnum_actual/median_3,Merchnum_total_3,Merchnum_actual/total_3,Merchnum_mean_7,Merchnum_actual/mean_7,Merchnum_max_7,Merchnum_actual/max_7,Merchnum_median_7,Merchnum_actual/median_7,Merchnum_total_7,Merchnum_actual/total_7,Merchnum_mean_14,Merchnum_actual/mean_14,Merchnum_max_14,Merchnum_actual/max_14,Merchnum_median_14,Merchnum_actual/median_14,Merchnum_total_14,Merchnum_actual/total_14,Merchnum_mean_30,Merchnum_actual/mean_30,Merchnum_max_30,Merchnum_actual/max_30,Merchnum_median_30,Merchnum_actual/median_30,Merchnum_total_30,Merchnum_actual/total_30,card_merchant_mean_0,card_merchant_actual/mean_0,card_merchant_max_0,card_merchant_actual/max_0,card_merchant_median_0,card_merchant_actual/median_0,card_merchant_total_0,card_merchant_actual/total_0,card_merchant_mean_1,card_merchant_actual/mean_1,card_merchant_max_1,card_merchant_actual/max_1,card_merchant_median_1,card_merchant_actual/median_1,card_merchant_total_1,card_merchant_actual/total_1,card_merchant_mean_3,card_merchant_actual/mean_3,card_merchant_max_3,card_merchant_actual/max_3,card_merchant_median_3

In [19]:
# final.to_csv('card_transactions_all_features.csv', index = False)

In [20]:
# # to export csv to google drive
# folder_link = 'https://drive.google.com/drive/folders/1iraC2IH38ybBVFi1BQQbN6rA2_rVmE7-'
# id = folder_link.split('/')[-1] # id: the drive file identifier
# print(id)

In [21]:
# # Uploading to Google Drive
# upload_file = 'card_transactions_clean.csv'
# gfile = drive.CreateFile({'parents': [{'id': id}]})
# # Read file and set it as the content of this instance.
# gfile.SetContentFile(upload_file)
# gfile.Upload() # Upload the file.

In [22]:
print('Time elapsed: ', dt.datetime.now() - start_time)

Time elapsed:  0:02:45.822074
